In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', quiet=True, numThreads=4)
import simuPOP as sim
import numpy as np
import pandas as pd
import random
import h5py
from saegus import analyze, breed, operators, parameters, parse

In [3]:
np.set_printoptions(precision=3, suppress=True)

In [ ]:
example_pop = sim.loadPopulation('example_pop.pop')

In [ ]:
example_pop.addInfoFields(['ind_id', 'mother_id', 'father_id', 'g', 'p'])

In [ ]:
sim.tagID(example_pop)

In [ ]:
tf = parse.TusonFounders()

In [ ]:
recom_map = tf.parse_recombination_rates('genetic_map.txt')

In [ ]:
founders = [[1, 2], [3, 4], [5, 6], [7, 8]]

In [ ]:
offspring_per_pair = 250

In [ ]:
magic = breed.MAGIC(example_pop, founders, recom_map)

In [ ]:
example_pop.popSize()

In [ ]:
magic.generate_f_one(founders, offspring_per_pair)

In [ ]:
random_cross = breed.RandomCross(example_pop, 4, 250)

In [ ]:
mothers, fathers = random_cross.converging_random_cross()

In [ ]:
chooser = breed.SecondOrderPairIDChooser(mothers, fathers)

In [ ]:
example_pop.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=recom_map)], numOffspring=1),
    subPopSize=1000),
    gen=1,
)

In [ ]:
final_random_cross = breed.RandomCross(example_pop, 2, 500)
mothers, fathers = final_random_cross.converging_random_cross()

In [ ]:
final_chooser = breed.SecondOrderPairIDChooser(mothers, fathers)

In [ ]:
example_pop.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(final_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=recom_map)], numOffspring=1),
    subPopSize=1000),
    gen=1,
)

In [ ]:
sim.stat(example_pop, numOfSegSites=sim.ALL_AVAIL, vars=['segSites'])

In [ ]:
qtl = sorted(random.sample(example_pop.dvars().segSites, 20))

In [ ]:
qtl

In [ ]:
trait = parameters.Trait()

In [ ]:
ae_table = trait.construct_allele_effects_table(example_pop, qtl, random.expovariate, 1)

In [ ]:
ae_array = trait.construct_ae_array(ae_table, qtl)

In [ ]:
ae_array[qtl]

In [2]:
integrated_data = h5py.File("example_integrated_data.hdf5")

In [ ]:
operators.calculate_g(example_pop, ae_array)

In [ ]:
np.array(example_pop.indInfo('g'))

In [ ]:
operators.calculate_error_variance(example_pop, 0.7)

In [ ]:
operators.calculate_p(example_pop)

In [ ]:
example_pop.dvars().epsilon

In [ ]:
print(np.array(example_pop.indInfo('p')))

In [ ]:
np.mean(example_pop.indInfo('g')), np.var(example_pop.indInfo('g'))

In [ ]:
np.mean(example_pop.indInfo('p')), np.var(example_pop.indInfo('p'))

In [ ]:
g = np.array(example_pop.indInfo('g'))